In [51]:
import numpy as np
import pandas as pd



def select_stratified_groups(data, strat_columns, group_size, weights=None, seed=None):
    """Подбирает стратифицированные группы для эксперимента.

    data - pd.DataFrame, датафрейм с описанием объектов, содержит атрибуты для стратификации.
    strat_columns - List[str], список названий столбцов, по которым нужно стратифицировать.
    group_size - int, размеры групп.
    weights - dict, словарь весов страт {strat: weight}, где strat - либо tuple значений элементов страт,
        например, для strat_columns=['os', 'gender', 'birth_year'] будет ('ios', 'man', 1992), либо просто строка/число.
        Если None, определить веса пропорционально доле страт в датафрейме data.
    seed - int, исходное состояние генератора случайных чисел для воспроизводимости
        результатов. Если None, то состояние генератора не устанавливается.

    return (data_pilot, data_control) - два датафрейма того же формата, что и data
        c пилотной и контрольной группами.
    """
    # YOUR_CODE_HERE
#
# Variant 2
#
    np.random.seed(seed)

    data_pilot = pd.DataFrame(data=None, columns=data.columns)
    data_control=pd.DataFrame(data=None, columns=data.columns)

    if weights is None:
        weights = data[strat_columns].value_counts()/len(data)

    # Пересчитываем веса в штуки и преобразуем {строка:число} в {(строка):число}
    #print(f"Пересчитываем веса в штуки")
    sizes = dict()
    for stratum, weight in weights.items():
        #print(f"{stratum=}, check={isinstance(stratum, str)}, {weight=}")
        size = np.array(weight * group_size).round().astype('int')
        if isinstance(stratum, (str, float, int)):
            # stratum - строка/number, преобразуем в iterable (tuple)
            key = tuple([stratum])
            sizes[key] = size
        else:
            sizes[stratum] = size 

    # Формируем группы
    for stratum, size in sizes.items():
        #print(f"\n{stratum=}, {size=}")
        mask = True
        for i, value in enumerate(stratum):
            #print(f"{i=}, {strat_columns[i]=}, {value=}")
            mask = mask & (data[strat_columns[i]] == value)
        df = data[mask]
        if (size != 0) and (len(df) >= 2*size):
            df = df.take(np.random.permutation(len(df))[:(2*size)])
            data_pilot = pd.concat([data_pilot, df[:size]])
            data_control = pd.concat([data_control, df[-size:]])
        else:
            # требуемый расчетный размер страты = 0, т.е. group_size слишком маленький
            # или
            # В data не хватает данных для формирования страты требуемого размера
            data_pilot = pd.DataFrame(data=None, columns=data.columns)
            data_control=pd.DataFrame(data=None, columns=data.columns)
            break;
        
    return data_pilot, data_control    

In [53]:
#select_stratified_groups(data, strat_columns, group_size, weights=None, seed=None)

data = pd.read_csv('lesson_4_hw_data.csv').rename(columns={'ОС':'OS'}).drop(columns=['group'])
data['c1'] = (data['inapp_prev_week'] > 0).astype('int')
data['c2'] = (data['inapp'] > 0).astype('int')
data[3] = data['c1']

strat_columns = ['c1'] #['OS','c1','c2']
group_size = 100 #5, 40
weights = None
#weights = {'android':0.6,'ios':0.4}
#weights = {0:0.95, 1:0.49}

seed = 15

data.head()

#print(data.dtypes)
#print(data[strat_columns].value_counts())

for group_size in [10,20,30,100,150]:
    data_pilot, data_control = select_stratified_groups(data=data, 
                                                    strat_columns=strat_columns, 
                                                    group_size=group_size, 
                                                    weights=weights, 
                                                    seed=seed)
    print(f"{group_size=}, {len(data_pilot)}, {len(data_control)}")

group_size=10, 0, 0
group_size=20, 20, 20
group_size=30, 30, 30
group_size=100, 100, 100
group_size=150, 150, 150


In [45]:
#
# Variant 2
#
np.random.seed(seed)

data_pilot = pd.DataFrame(data=None, columns=data.columns)
data_control=pd.DataFrame(data=None, columns=data.columns)

if weights is None:
    weights = data[strat_columns].value_counts()/len(data)

# Пересчитываем веса в штуки и преобразуем {строка:число} в {(строка):число}
print(f"Пересчитываем веса в штуки")
sizes = dict()
for stratum, weight in weights.items():
    #print(f"{stratum=}, check={isinstance(stratum, str)}, {weight=}")
    size = np.array(weight * group_size).round().astype('int')
    if isinstance(stratum, (str, float, int)):
        # stratum - строка, преобразуем в iterable (list)
        key = tuple([stratum])
        sizes[key] = size
    else:
        sizes[stratum] = size 

# Формируем группы
for stratum, size in sizes.items():
    print(f"\n{stratum=}, {size=}")
    mask = True
    for i, value in enumerate(stratum):
        print(f"{i=}, {strat_columns[i]=}, {value=}")
        mask = mask & (data[strat_columns[i]] == value)
    df = data[mask]
    if (size != 0) and (len(df) >= 2*size):
        df = df.take(np.random.permutation(len(df))[:(2*size)])
        data_pilot = pd.concat([data_pilot, df[:size]])
        data_control = pd.concat([data_control, df[-size:]])
    else:
        # требуемый расчетный размер страты = 0, т.е. group_size слишком маленький
        # или
        # В data не хватает данных для формирования страты требуемого размера
        data_pilot = pd.DataFrame(data=None, columns=data.columns)
        data_control=pd.DataFrame(data=None, columns=data.columns)
        break;
        
sizes, data_control, data_pilot

Пересчитываем веса в штуки

stratum=(0,), size=95
i=0, strat_columns[i]='c1', value=0

stratum=(1,), size=49
i=0, strat_columns[i]='c1', value=1


({(0,): 95, (1,): 49},
            OS age inapp_prev_week inapp c1 c2  3
 1943  android  38               0     0  0  0  0
 1900  android  37               0     0  0  0  0
 1969      ios  38               0     0  0  0  0
 538   android  40               0     0  0  0  0
 1399  android  38               0     0  0  0  0
 ...       ...  ..             ...   ... .. .. ..
 8     android  37             100   120  1  1  1
 642       ios  20             130   110  1  1  1
 604       ios  32              40   110  1  1  1
 262       ios  33              50    70  1  1  1
 577   android  34             130   120  1  1  1
 
 [144 rows x 7 columns],
            OS age inapp_prev_week inapp c1 c2  3
 762       ios  32               0     0  0  0  0
 1681  android  16               0     0  0  0  0
 1108  android  32               0     0  0  0  0
 1573  android  21               0     0  0  0  0
 1487  android  23               0     0  0  0  0
 ...       ...  ..             ...   ... .. .. ..


In [ ]:
#
# Variant 1
#

np.random.seed(seed)

data_pilot = pd.DataFrame(data=None, columns=data.columns)
data_control=pd.DataFrame(data=None, columns=data.columns)

if weights is None:
    weights = data[strat_columns].value_counts()/len(data)

# Пересчитываем веса в штуки
sizes = dict()
for stratum, weight in weights.items():
    sizes[stratum] = np.array(weight * group_size).round().astype('int')

# Проверяем реализуемость стратификации
if (np.array([i for i in sizes.values()]).sum() != group_size) \
    or 2*group_size > len(data) \
    or len([i for i in sizes.values() if i == 0]) != 0:
    print(f"ERROR: {group_size=}, {sizes.values()}, {np.array([i for i in sizes.values()]).sum()}, {len([i for i in sizes.values() if i == 0])}")
    # return data_pilot, data_control
else:
    print(f"SUCCESS: {group_size=}")

# Формируем группы
for stratum, size in sizes.items():
    print(f"\n{stratum=}, {size=}")
    mask = True
    for i, value in enumerate(stratum):
        print(f"{i=}, {strat_columns[i]=}, {value=}")
        mask = mask & (data[strat_columns[i]] == value)
    df = data[mask].copy()
    df = df.take(np.random.permutation(len(df))[:(2*size)])
    data_pilot = pd.concat([data_pilot, df[:size]])
    data_control = pd.concat([data_control, df[-size:]])
        

In [ ]:
data_pilot[strat_columns].value_counts()/len(data_pilot)